# 데이터 정리

In [1]:
import pandas as pd
import numpy as np

# 데이터 불러오고 병합하기
retail1 = pd.read_excel('online_retail_2.xlsx', sheet_name='Year 2009-2010')
retail2 = pd.read_excel('online_retail_2.xlsx', sheet_name='Year 2010-2011')
retail = pd.concat([retail1, retail2], axis=0)

retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [2]:
retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1067371 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1067371 non-null  object        
 1   StockCode    1067371 non-null  object        
 2   Description  1062989 non-null  object        
 3   Quantity     1067371 non-null  int64         
 4   InvoiceDate  1067371 non-null  datetime64[ns]
 5   Price        1067371 non-null  float64       
 6   Customer ID  824364 non-null   float64       
 7   Country      1067371 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 73.3+ MB


## 데이터 평가
**1. Description과 Customer ID에서 NULL값이 존재.**
  
- 하지만 Description이 NULL이더라도 StockCode가 존재하므로, StockCode로 식별이 가능하다.
- 만약 Description값 활용이 필요하다면 같은 StockCode에 대한 Description값을 찾아 NULL값을 채워 활용할 수 있음.

**2. Customer ID는 고객 식별자이므로, NULL값인 행은 제거해도 무방.**

In [3]:
retail.describe()

,Quantity,Price,Customer ID
count,1.067371e+06,1.067371e+06,824364.000000
mean,9.938898e+00,4.649388e+00,15324.638504
std,1.727058e+02,1.235531e+02,1697.464450
min,-8.099500e+04,-5.359436e+04,12346.000000
25%,1.000000e+00,1.250000e+00,13975.000000
50%,3.000000e+00,2.100000e+00,15255.000000
75%,1.000000e+01,4.150000e+00,16797.000000
max,8.099500e+04,3.897000e+04,18287.000000


**3. Quantity와 Price에 음수값이 존재. 이는 잘못된 데이터이므로 제거해야함.**

In [4]:
# Customer Id가 Null인 데이터 제거
retail = retail[pd.notnull(retail['Customer ID'])]

# Quantity가 0보다 큰 데이터만 남김
retail = retail[retail['Quantity']>0]

# Price가 0보다 큰 데이터만 남김
retail = retail[retail['Price']>0]

retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 805549 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      805549 non-null  object        
 1   StockCode    805549 non-null  object        
 2   Description  805549 non-null  object        
 3   Quantity     805549 non-null  int64         
 4   InvoiceDate  805549 non-null  datetime64[ns]
 5   Price        805549 non-null  float64       
 6   Customer ID  805549 non-null  float64       
 7   Country      805549 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 55.3+ MB


➕ Description열의 데이터를 따로 건들지 않았음에도, 앞선 과정에서 Null값이 제거되었음을 확인.

In [5]:
retail.describe()

,Quantity,Price,Customer ID
count,805549.000000,805549.000000,805549.000000
mean,13.290522,3.206561,15331.954970
std,143.634088,29.199173,1696.737039
min,1.000000,0.001000,12346.000000
25%,2.000000,1.250000,13982.000000
50%,5.000000,1.950000,15271.000000
75%,12.000000,3.750000,16805.000000
max,80995.000000,10953.500000,18287.000000


➕ 앞으로의 분석에서 각 행의 Price * Quantity로 주문 금액은 자주 사용될 것이므로, 이를 새로운 컬럼으로 추가.

In [6]:
# 주문금액 컬럼 추가
retail['CheckoutPrice'] = retail['Quantity'] * retail['Price']
retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,CheckoutPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0


**4. 2010-12-01 ~ 2010-12-09까지의 데이터가 중복되어 존재하는 것을 확인할 수 있으므로 중복된 데이터를 제거.**

**5. 2009-12-01 ~ 2011-12-09까지의 데이터가 존재하므로, 편의상 2년치 데이터만 사용하기 위해 2011-12 데이터를 제거.**

In [7]:
# 중복된 데이터 제거
retail.drop_duplicates(inplace=True)
retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 779425 entries, 0 to 541909
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Invoice        779425 non-null  object        
 1   StockCode      779425 non-null  object        
 2   Description    779425 non-null  object        
 3   Quantity       779425 non-null  int64         
 4   InvoiceDate    779425 non-null  datetime64[ns]
 5   Price          779425 non-null  float64       
 6   Customer ID    779425 non-null  float64       
 7   Country        779425 non-null  object        
 8   CheckoutPrice  779425 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 59.5+ MB


In [8]:
# InvoiceDate에서 2011년 12월 데이터 제거
retail = retail[retail['InvoiceDate'] < '2011-12-01']
retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 762398 entries, 0 to 516368
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Invoice        762398 non-null  object        
 1   StockCode      762398 non-null  object        
 2   Description    762398 non-null  object        
 3   Quantity       762398 non-null  int64         
 4   InvoiceDate    762398 non-null  datetime64[ns]
 5   Price          762398 non-null  float64       
 6   Customer ID    762398 non-null  float64       
 7   Country        762398 non-null  object        
 8   CheckoutPrice  762398 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 58.2+ MB


In [9]:
# csv 파일로 저장
retail.to_csv('OnlineRetailClean.csv')